In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import *

1.8.1
0.8.0a0+e4e171a


In [68]:
cfg = Config(
    noise_nsr_dbs = [20, 10, 3],
    site='SSW', 
    use_neptune=True, 
    n_epochs=10,
    bs=32, 
    lr=1e-3, 
    model='densenet-mixedspec',
    # noise_dir = 'data/birdclef-2021/train_soundscapes.wav',
)

In [69]:
cfg.as_dict()

{'site': 'SSW',
 'data_dir': 'data/birdclef-2021',
 'checkpoint_dir': 'checkpoints',
 'bs': 32,
 'n_workers': 12,
 'training_dataset_size': 10752,
 'duration': 5,
 'use_noise': True,
 'noise_nsr_dbs': [20, 10, 3],
 'noise_dir': 'data/noise/BirdVox-DCASE-20k/wav-32k',
 'use_overlays': True,
 'max_overlays': 5,
 'overlay_weights': [0.71986223,
  0.21010333,
  0.06314581,
  0.00574053,
  0.00114811],
 'overlay_snr_dbs': [20, 10, 3],
 'use_neptune': True,
 'sr': 32000,
 'n_frames': 160000,
 'n_fft': 512,
 'window_length': 512,
 'n_mels': 64,
 'hop_length': 312,
 'f_min': 150,
 'f_max': 15000,
 'lr': 0.001,
 'n_epochs': 10,
 'model': 'densenet-mixedspec',
 'optimizer': 'torch.optim.Adam',
 'loss_fn': 'torch.nn.BCEWithLogitsLoss',
 'scheduler': 'torch.optim.lr_scheduler.CosineAnnealingLR',
 'labels': ['amecro',
  'amegfi',
  'amerob',
  'balori',
  'belkin1',
  'bkcchi',
  'blujay',
  'cangoo',
  'chswar',
  'comgra',
  'comyel',
  'dowwoo',
  'eastow',
  'eawpew',
  'gockin',
  'grycat',
  

In [70]:
prep = torch.nn.Sequential(
    MixedSig2Spec(cfg),
)

main_model = torch.hub.load('pytorch/vision:v0.9.0', 'densenet121', pretrained=True)
main_model.classifier = torch.nn.Linear(in_features=1024, out_features=len(cfg.labels), bias=True)

model = torch.nn.Sequential(prep, main_model)
for param in model.parameters():
    param.requires_grad = False

for m in [model[1].features.denseblock3, model[1].features.transition3, model[1].features.denseblock4, model[1].features.norm5, model[1].classifier]:
    for param in m.parameters():
        param.requires_grad = True
    



model = model.cuda()

Using cache found in /home/koen/.cache/torch/hub/pytorch_vision_v0.9.0


In [71]:
lrn = Learner("densenet-densenet-mixed-spec-frozen", cfg, model)

In [72]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-44


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:   9 train loss: 0.121960 train f1: 0.505074 valid loss: 0.119725 valid f1: 0.067764


  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:  10 train loss: 0.121431 train f1: 0.518413 valid loss: 0.136617 valid f1: 0.062706


In [74]:
for param in model.parameters():
    param.requires_grad = True

lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-45


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:   9 train loss: 0.102703 train f1: 0.619460 valid loss: 0.101777 valid f1: 0.302725


  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:  10 train loss: 0.103035 train f1: 0.620997 valid loss: 0.109876 valid f1: 0.205080


In [73]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,127,30177,2653,643,0.071549
2,0.2,77,31637,1193,693,0.075490
3,0.3,56,32082,748,714,0.071156
4,0.4,44,32284,546,726,0.064706
5,0.5,38,32426,404,732,0.062706
6,0.6,33,32495,335,737,0.057996
7,0.7,24,32559,271,746,0.045070
8,0.8,18,32644,186,752,0.036961
9,0.9,10,32746,84,760,0.023148


In [51]:
lrn.load_checkpoint()

{'epoch': 17,
 'valid_loss': 0.08831553920907409,
 'valid_score': 0.2621259093284607}

In [52]:
lrn.evaluate()

  0%|          | 0/38 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,770,0,32830,0,0.044807
1,0.1,329,31856,974,441,0.317414
2,0.2,238,32523,307,532,0.361977
3,0.3,193,32669,161,577,0.343416
4,0.4,160,32731,99,610,0.310982
5,0.5,127,32758,72,643,0.262126
6,0.6,98,32779,51,672,0.213275
7,0.7,78,32797,33,692,0.177072
8,0.8,57,32809,21,713,0.134434
9,0.9,37,32819,11,733,0.090465
